In [1]:
import os
import pandas as pd

In [2]:
print(os.getcwd())

c:\Users\facua\Master in Data Science\Research\ChlorineProject\ChlorineProject\notebooks


In [3]:
import os

# Get the current working directory
current_dir = os.getcwd()
print("Current Directory:", current_dir)

# Go back one folder
parent_dir = os.path.dirname(current_dir)
print("Parent Directory:", parent_dir)

# Change the current working directory to the parent directory
os.chdir(parent_dir)
print("Changed to Parent Directory:", os.getcwd())


Current Directory: c:\Users\facua\Master in Data Science\Research\ChlorineProject\ChlorineProject\notebooks
Parent Directory: c:\Users\facua\Master in Data Science\Research\ChlorineProject\ChlorineProject
Changed to Parent Directory: c:\Users\facua\Master in Data Science\Research\ChlorineProject\ChlorineProject


In [4]:
!pip install -r requirements.txt

In [5]:
!pip install datasets

   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   --------------------------------------  542.7/547.8 kB 17.2 MB/s eta 0:00:01
   --------------------------------------- 547.8/547.8 kB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/369.0 kB ? eta -:--:--
   --------------------------------------- 369.0/369.0 kB 23.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   - -------------------------------------- 0.8/25.8 MB 26.4 MB/s eta 0:00:01
   -- ------------------------------------- 1.6/25.8 MB 20.5 MB/s eta 0:00:02
   --- ------------------------------------ 2.5/25.8 MB 20.3 MB/s eta 0:00:02
   ----- ---------------------------------- 3.3/25.8 MB 19.0 MB/s eta 0:00:02
   ------ --------------------------------- 4.1/25.8 MB 18.8 MB/s eta 0:00:02
   ------- ---

In [6]:
!pip install accelerate -U

   ---------------------------------------- 0.0/309.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/309.4 kB ? eta -:--:--
   --------------- ------------------------ 122.9/309.4 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 309.4/309.4 kB 3.2 MB/s eta 0:00:00


In [4]:
!python finetune/seqcls/run_seqcls_gpt.py finetune/seqcls/config.json

^C


07/15/2024 10:55:42 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0distributed training: True, 16-bits training: False

label_list ['maybe', 'no', 'yes']
Adding [PAD] token to tokenizer and model word embeddings.
{'loss': 2.236, 'grad_norm': 185.846435546875, 'learning_rate': 8.888888888888888e-06, 'epoch': 0.22}
{'loss': 2.7352, 'grad_norm': 207.66424560546875, 'learning_rate': 1.7777777777777777e-05, 'epoch': 0.44}
{'loss': 2.0797, 'grad_norm': 0.2045285552740097, 'learning_rate': 1.3333333333333333e-05, 'epoch': 0.67}
{'loss': 2.8386, 'grad_norm': 0.9423865675926208, 'learning_rate': 4.444444444444444e-06, 'epoch': 0.89}
{'train_runtime': 10143.7559, 'train_samples_per_second': 0.044, 'train_steps_per_second': 0.044, 'train_loss': 2.4267981804741754, 'epoch': 1.0}
***** train metrics *****
  epoch                    =        1.0
  total_flos               =   109514GF
  train_loss               =     2.4268
  train_runtime            = 2:49:03.75
  train_samples       

c:\Users\facua\anaconda3\envs\chlorine\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(

Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 450 examples [00:00, 8812.96 examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]
Generating validation split: 50 examples [00:00, 4913.20 examples/s]

Generating test split: 0 examples [00:00, ? examples/s]
Generating test split: 500 examples [00:00, 29889.29 examples/s]
c:\Users\facua\anaconda3\envs\chlorine\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[WARNING|modeling_utils.py:4282] 2024-07-15 10:55:44,375 >> Some weights of GP

In [ ]:
import json
import csv


# Model checkpoint directory
output_dir = "runs/pubmedqa_hf/GPT2"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForSequenceClassification.from_pretrained(output_dir)

# model in eval mode
model.eval()

# Directory of the JSON file with the test data
input_json_path = "ChlorineProject/pubmedqa_hf/test.json"
output_csv_path = "predictions_output.csv"

# id2label mapping
id2label = {0: "maybe", 1: "no", 2: "yes"}

# Read Json file
with open(input_json_path, 'r') as infile:
    data = [json.loads(line) for line in infile]

# Run predictions and save to CSV
with open(output_csv_path, mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    
    # Write the header
    header = ["id", "prediction", "prediction_label"] + [f"prob_{id2label[i]}" for i in range(model.config.num_labels)] + ["true_label"]
    writer.writerow(header)
    
    for item in data:
        id_number = item["id"]
        sentence1 = item["sentence1"]
        sentence2 = item["sentence2"]
        true_label = item.get("label", "N/A")
        
        # Tokenize the input
        inputs = tokenizer(sentence1, sentence2, return_tensors="pt", padding=True, truncation=True, max_length=128)
        
        # Prediction
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=-1).squeeze().tolist()
            prediction = torch.argmax(logits, dim=-1).item()
            prediction_label = id2label[prediction]
        
        # Write the row
        row = [id_number, prediction, prediction_label] + probabilities + [true_label]
        writer.writerow(row)

print(f"Predicciones guardadas en {output_csv_path}")